# GlobeStream: Simulation of an Interactive Video Platform

You will find below the code for the building blocks of the simulated video platform, the head function made up of all these blocks, and finally a call to the head function itself that allows you to directly dive in the programm. The successive development steps and their purposes are thoroughly explained. If you started by opening this notebook, feel free to consult as well the associated 'readme' file that gives more details about the followed logic. Hope you will enjoy going through this code!

In [1]:
from datetime import datetime
import time
import csv
import random

## Addition of a Video to GlobeStream

When adding a video, the user fills a certain part of the new record ('title', 'category', 'duration') while the rest is automatically created in the background. The 'publish' date attribute is the current datetime, 'days_online' and 'viewed' are of course starting at 0 (refreshed when running the 'refresh_videos_job'). The unique identifier is made equal to the starting nb. of rows of the 'videos' file: since as I noticed each addition of a record creates also an empty line, 'id' attributed in this way will be equal to the previous one plus 2. It actually doesn't really matter if the 'id' entries are not spaced one by one, as long as they are unique. For this project I imagine no video can be removed therefore there is no risk that removing a record and setting id for a new one based on the new reduced length will conflict with a pre-existing record.

#### Helper function: convert duration input to number with decimals

First thing, I design a helper function that converts the duration entered by the user into a decimal number expressing the nb. of minuts. This is intended to later on ease the advanced search process in which the user enters a min. and max. duration in minuts

In [2]:
def convert_minuts_decimal(duration):
    if len(duration.split(':')) ==3:
        hours = int(duration.split(':')[0])*60
        mins = int(duration.split(':')[1])
        secs = round(int(duration.split(':')[2])/60,2)
        return hours+mins+secs
    elif len(duration.split(':')) ==2:
        mins = int(duration.split(':')[0])
        secs = round(int(duration.split(':')[1])/60,2)
        return mins+secs

print(convert_minuts_decimal('0:17'))
    

0.28


#### Main Add Function

In [3]:
def add_video(title,category,duration):
    with open('videos.csv') as videos:
        len = sum(1 for row in videos)
    with open('videos.csv', 'a') as videos:
        fields = ['id','title', 'category', 'duration', 'publish_date', 'days_online', 'viewed']
        output_writer = csv.DictWriter(videos, fieldnames= fields)        
        output_writer.writerow({'id': len,'title': title, 'category': category, 'duration': convert_minuts_decimal(duration), 'publish_date': datetime.now(),\
                                'days_online':0, 'viewed':0})
    print('Video added! Thanks for your contribution!')

## Refresh the Videos Background File

The below coded function deals with two types of video metadata that are time-sensitive:

1) the nb. of days that elapsed since the video was uploaded ('days_online'): it is done by substracting the datetime of when this happened from the '.now()' datetime of when the refresh takes place
2) the nb. of times the video was viewed overall ('viewed') : this is done by taking the 'viewed' number as it is currently in the file and by incrementing it using both a) the number of days since the last refresh multiplied by b) the average number of daily views during this period, which is determined by a function returning a random integer between 3000 and 70000

In [4]:
def refresh_videos_job():
    # 1) download current videos data and store them in temp list 'records'
    with open('videos.csv', newline = "") as unrefreshed_videos:
        videos_dict = csv.DictReader(unrefreshed_videos)
        records = []
        for row in videos_dict:        
            records.append(row)
    for video in records:
        # 2) refresh nb. of views
        now = datetime.now()
        publish = datetime.strptime(video['publish_date'], '%Y-%m-%d %H:%M:%S.%f')
        refresh_lapse = ((now-publish).days) -int(video['days_online'])
        viewed_new = random.randint(3000,70000)*refresh_lapse
        video['viewed'] = int(video['viewed'])+viewed_new
        # 3) refresh nb. of days online
        video['days_online'] = (now-publish).days
    # 4) erase initial records with the refreshed ones
    with open('videos.csv', 'w') as videos_refresh:
        fields = ['id','title', 'category', 'duration', 'publish_date', 'days_online', 'viewed']
        output_writer = csv.DictWriter(videos_refresh, fieldnames = fields)
        output_writer.writeheader()
        for video in records:
            output_writer.writerow(video)
    


## Search & Watch Videos

User will have the possibility to make either an advanced search, being prompted for nearly all the possible types of metadata, or to just type in some text elements that will be matched with the in-store videos' titles. For this I will first store the records in a temporary list. I assume that the downloaded records were already subject to the 'refresh_videos_job' function and therefore have up to date 'days_online' and 'viewed' attributes.

Based on the results returned by the search function, a watch function will allow the user to simulate chosing one of the results based on the video 'id' and watch it. Of course it will not launch any video whatsoever, we will just imagine that by chosing one number the user watches the video. 

Neither will I, for the sake of keeping the complexity of functions in reasonable boundaries, automatically add one to the views count of the watched video in the file. Making again a call to our imaginations, let's pretend that views count is incremented in bulk every some time period.

#### Conditions-Checking Helper Functions

Helper functions that check dictionaries from a list of dictionaries and select according to a criteria and an operator. If any of the helper functions used successively for an advanced search returns 'False', the search will stop because it will mean that no result matches ALL the criteria. 

Note that the input from which the function takes records to check varies: it is because when checking for a criteria, we can either be in the case of restricting some selection that can't grow anymore, or to look in the whole set of records. 

The first case is when we had already some condition checks made and a list of results still needing to be checked against the following criteria was built: in this potentiality we check in the input 'selection'. 

The second case is either when we are starting the check of the conditions or when the user refused to do a check versus the previous criteria proposed (which will be possible, by pressing 'x'). In which case the condition-checking function applies to the whole set of videos records.

In [5]:
import operator

def append_criteria_equal(field,criteria, videos, selection):
        if selection == []:
            for video in videos:
                if criteria.upper() in video[field].upper():
                    selection.append(video)
        else:
            selection = [video for video in selection if criteria in video[field]]
        return selection != []
    
def append_criteria_between(criteria, videos, selection):
        if selection == []:
            for video in videos:
                if float(video['duration']) >= float(criteria[0]) and float(video['duration']) <= float(criteria[1]):
                    selection.append(video)
        else:
            selection = [video for video in selection if float(video['duration']) >= float(criteria[0]) and float(video['duration']) <= float(criteria[1])]
        return selection != []


def append_criteria_superior(field,criteria, videos, selection):
        if selection == []:
            for video in videos:
                if float(video[field]) >= criteria:
                    selection.append(video)
        else:
            selection = [video for video in selection if float(video[field])>= float(criteria)]
        return selection != []
    
def append_criteria_inferior(field,criteria, videos, selection):
        if selection == []:
            for video in videos:
                if float(video[field]) <= criteria:
                    selection.append(video)
        else:
            selection = [video for video in selection if float(video[field])<= float(criteria)]
        return selection != []

#### The Search Function

In [6]:
def search_video(Advanced = False):
    with open('videos.csv') as videos:
        videos_dict = csv.DictReader(videos)
        records = []
        for row in videos_dict:
            records.append(row)
    if Advanced:
        advanced_search_return = []
        print('\nADVANCED SEARCH. IF NO PREFERENCE FOR THE GIVEN CRITERIA JUST PRESS X')
        # series of inputs the user has to enter and that get check with the appropriate above-coded helper function
        kw = input('Enter keyword(s): ')
        if kw.upper() != 'X':
            if append_criteria_equal('title', kw, records, advanced_search_return) == False:
                print('No results found!')
                return None
        cat = input('Choose category from Fun/News/Sport:')
        if cat.upper() != 'X':
            if append_criteria_equal('category', cat, records, advanced_search_return) == False:
                print('No results found!')
                return None
        duration_min = input('Choose min. duration (minuts):')
        duration_max = input('Choose max. duration (minuts):')
        # possible to enter 0,1 or 2 boundaries for the duration of the video
        if duration_min.upper() != 'X' and duration_max.upper() != 'X':
            if append_criteria_between([duration_min, duration_max], records, advanced_search_return) == False:
                print('No results found!')
                return None
        elif duration_min.upper() != 'X' and duration_max.upper() == 'X':
            if append_criteria_superior('duration',duration_min, records, advanced_search_return)== False:
                print('No results found!')
                return None
        elif duration_min.upper() == 'X' and duration_max.upper() != 'X':
            if append_criteria_inferior('duration',duration_max, records, advanced_search_return)== False:
                print('No results found!')
                return None
        recentness = input('Not older than how many days:')
        if recentness.upper() != 'X':
            if append_criteria_inferior('days_online',recentness, records, advanced_search_return)== False:
                print('No results found!')
                return None     
        popularity = input('At least how many views:')
        if popularity.upper() != 'X':
            if append_criteria_superior('viewed',popularity, records, advanced_search_return)== False:
                print('No results found!')
                return None
        for video in advanced_search_return:
                print('\n')
                print(video['id']+':'+video['title'])
                print('Category: '+video['category'])
                duration_split = video['duration'].split('.')
                duration_split = [float(t) for t in duration_split]
                if duration_split[0] >= 60:
                    print(f'Duration: {int(duration_split[0]//60)}:{int(duration_split[0]%60)}:{int((duration_split[1]*0.6))}')
                else:
                    print(f'Duration: {int(duration_split[0])}:{int((duration_split[1]*0.6))}')                  
                print('Uploaded '+ video['days_online']+ ' days ago')
                print(video['viewed'] +' views')
        return advanced_search_return

    else:
        title = input('\nType what you search for and press Enter: ')
        basic_search_return = []
        for video in records:
            if (title.upper() in video['title'].upper()):
                print('\n')
                print(video['id']+':'+video['title'])
                print('Category: '+video['category'])  
                duration_split = video['duration'].split('.')
                duration_split = [float(t) for t in duration_split]
                if duration_split[0] >= 60:
                    print(f'Duration: {int(duration_split[0]//60)}:{int(duration_split[0]%60)}:{int((duration_split[1]*0.6))}')
                else:
                    print(f'Duration: {int(duration_split[0])}:{int((duration_split[1]*0.6))}')                  
                print('Uploaded '+ video['days_online']+ ' days ago')
                print(video['viewed'] +' views')                
                basic_search_return.append(video)
        if basic_search_return == []:
            print('No results found!')
            return None
        else:
            return basic_search_return
    
            
        
    

####  The Watch Function

The input of this function is going to be the results that the search function returned, allowing to make a choice between them.

In [7]:
def watch_video(videos):
    id = input('Type the id of the video you want to watch (close search press X): ')
    if id.upper() == 'X':
        return
    video = [vid for vid in videos if vid['id']==id]
    print('\nLoading '+'\''+video[0]['title']+'\'')
    print('\nWatching '+'\''+video[0]['title']+'\'')
    print('\nThe End! Hope you enjoyed watching '+'\''+video[0]['title']+'\'')

## Assemblying the Building Blocks

Now that we have a function to refresh the background data, a function to add a video, a function to search videos, a function to watch a video, all we need to do is to make these functions work together in a coherent program that streamlines the experience of a user while connected on a video platform. This is achieved thanks to the below coded function.

Note the recursive approach through which after each action of adding or watching a video, the function calls itself again, allowing the user to simply restart from front page, and over, until the program gets stopped somehow. However while the initial call triggers a refresh of the base of videos, the following don't because it's useless to repeat it.

In [8]:
def GlobeStream(refresh = True):
    if refresh == True:
        refresh_videos_job()
    choice = input('\nHello! Add a video (+) or search for one (s): ' )
    if choice == '+':
        add_video(input('\nVideo title: '), input('Video category from Fun/News/Sport: '), input('Enter video duration (mm:ss or hh:mm:ss) '))
        GlobeStream(refresh = False)
    else:
        search_type = input('Choose advanced (a) or simple (s) search: ')
        if search_type.upper() == 'A':
            result = search_video(Advanced = True)
            if result:
                watch_video(result)
                GlobeStream(refresh = False)
            else:
                GlobeStream(refresh = False)                    
        else:
            result = search_video(Advanced = False)
            if result:
                watch_video(result)
                GlobeStream(refresh = False)
            else:
                GlobeStream(refresh = False) 
            
            
            

In [ ]:
GlobeStream()